# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hawaiian paradise park,19.5933,-154.9731,23.91,92,53,3.94,US,1693724200
1,1,megion,61.0296,76.1136,16.01,50,42,7.01,RU,1693724201
2,2,piacabucu,-10.4056,-36.4344,23.21,85,42,4.17,BR,1693724151
3,3,waitangi,-43.9535,-176.5597,8.23,81,97,5.56,NZ,1693724202
4,4,lebu,-37.6167,-73.6500,7.55,76,100,6.29,CL,1693724202


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=1.5,
    color="City",
    hover_cols=["Lng", "Lat", "City", "Humidity"],
)
# Display the map
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
Ideal_cities_df = city_data_df.query("Humidity >= 30").query("Cloudiness <= 70")
# Drop any rows with null values
Ideal_cities_df.dropna(axis=0, inplace=True)

# Display sample data
Ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hawaiian paradise park,19.5933,-154.9731,23.91,92,53,3.94,US,1693724200
1,1,megion,61.0296,76.1136,16.01,50,42,7.01,RU,1693724201
2,2,piacabucu,-10.4056,-36.4344,23.21,85,42,4.17,BR,1693724151
7,7,blackmans bay,-43.0167,147.3167,15.41,55,0,4.74,AU,1693724205
8,8,pankovka,58.5000,31.1999,16.69,75,67,2.34,RU,1693724205


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(
    {
        "City": Ideal_cities_df["City"],
        "Country": Ideal_cities_df["Country"],
        "Lat": Ideal_cities_df["Lat"],
        "Lng": Ideal_cities_df["Lng"],
        "Humidity": Ideal_cities_df["Humidity"],
    }
)
hotel_df = hotel_df.reset_index(drop=True)
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,hawaiian paradise park,US,19.5933,-154.9731,92,
1,megion,RU,61.0296,76.1136,50,
2,piacabucu,BR,-10.4056,-36.4344,85,
3,blackmans bay,AU,-43.0167,147.3167,55,
4,pankovka,RU,58.5000,31.1999,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
radius = 10000
limit  = 20

params = {"apiKey": geoapify_key,
         "limit": limit,
         "categories":"accommodation.hotel"}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df["Lat"][index]
    longitude = hotel_df["Lng"][index]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"][
            "name"
        ]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(
        f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}"
    )

# Display sample data
hotel_df

Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
megion - nearest hotel: Адрия
piacabucu - nearest hotel: O Leão
blackmans bay - nearest hotel: Villa Howden
pankovka - nearest hotel: отель "На Панковке"
albany - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
palma sola - nearest hotel: No hotel found
cangzhou - nearest hotel: No hotel found
machico - nearest hotel: Residence Inn
edinburgh of the seven seas - nearest hotel: No hotel found
prado - nearest hotel: Pousada Recanto do Prado
kongolo - nearest hotel: No hotel found
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
georgetown - nearest hotel: Page 63 hostel
thompson - nearest hotel: Thompson Inn
puerto natales - nearest hotel: Dorotea Patagonia Hostel
sur - nearest hotel: Sur Hotel
inhambane - nearest hotel: Hotel de Inhambane
santana - nearest hotel: Hotel Teodoro
punta arenas - nearest hotel: Hotel Lacolet
marquette - nearest hotel: No hotel found
sorsk - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
0,hawaiian paradise park,US,19.5933,-154.9731,92,No hotel found
1,megion,RU,61.0296,76.1136,50,Адрия
2,piacabucu,BR,-10.4056,-36.4344,85,O Leão
3,blackmans bay,AU,-43.0167,147.3167,55,Villa Howden
4,pankovka,RU,58.5000,31.1999,75,"отель ""На Панковке"""
...,...,...,...,...,...,...
265,dubrovnik,HR,42.6481,18.0922,47,Bellevue
266,cadale,SO,2.7603,46.3222,76,No hotel found
267,majene,ID,-3.5403,118.9707,57,No hotel found
268,balsas,BR,-7.5325,-46.0356,36,Imperial Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_data_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=1.5,
    color="City",
    hover_cols=["Lng", "Lat", "City", "Humidity", "Hotel Name", "Country"],
)

# Display the map
hotel_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)